# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [5]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [6]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [9]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating Keyspace

In [10]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}    
    """
    )
    
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:

try:
    session.set_keyspace('udacity')
    
except Exception as e:
    print(e)
    

### Now we will create tables to run the following queries.


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




###  Here we are creating a table with Primary Key containing sessionid as partition key and iteminsession as clustering key. Partitioning is done by session id and in that partition, rows are ordered by iteminsession

In [16]:


query = "CREATE TABLE IF NOT EXISTS session_history"

query = query + "(sessionid INT, iteminsession INT, artist TEXT,\
      length FLOAT,  song TEXT,\
      PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
    
except Exception as e:
    print(e)
                    

In [19]:
# Here we are inserting data from the csv file to our table.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionid,iteminsession, artist, \
      length,  song)"
        query = query + "VALUES(%s,%s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

### Using SELECT statement to get  the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [24]:


query = "SELECT artist, song, length \
    FROM session_history\
    WHERE sessionid =338 AND iteminsession = 4 ;"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.length, row.song)

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


### Here we are creating a table with Primary key containing userid and session id as Partitioning Key and iteminsession as clustering key. Partitioning is done by userid and session id and the within the partioning, rows are sorted in order by itemin session.

In [31]:


query = "CREATE TABLE IF NOT EXISTS song_playlist_session"

query = query + "(userid INT,  sessionid INT, iteminsession INT, artist TEXT,\
      song TEXT,firstname TEXT, lastname TEXT, \
      PRIMARY KEY ((userid, sessionid), iteminsession))"

try:
    session.execute(query)
    
except Exception as e:
    print(e)

    
                    

In [32]:
## Inserting data to our table from csv file.

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userid, sessionid, iteminsession, artist, song,\
                 firstname, lastname)"
        query = query + "VALUES(%s,%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4],  ))

### Using SELECT statement to get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [34]:
query = "SELECT artist, song, firstname, lastname \
    FROM song_playlist_session \
    WHERE  userid =10 AND sessionid = 182 ;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Here we are creating a table with Primary key containing song as Partitioning Key and user id as clustering key. Partitioning is done by song and the within the partioning, rows are sorted in order by user id.

In [35]:
query = "CREATE TABLE IF NOT EXISTS song_user_data"
query = query + "(song TEXT, userid INT, firstname TEXT, lastname TEXT, \
      PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [37]:
## Inserting data to our table from CSV file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_data (song, userid, firstname, lastname)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### Using SELECT statement to get user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

In [38]:

query = "SELECT firstname, lastname\
    FROM song_user_data\
    WHERE  song = 'All Hands Against His Own' ;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [39]:
query = "drop table session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_user_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()